In [ ]:
#!/usr/bin/python
# -*- coding: latin-1 -*-
import os

try :
    import pywikibot as pb

except :
    import sys, inspect
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[0:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    
import pandas as pd
from mako.template import Template

from io import StringIO

In [ ]:
YEAR                = 2017
TAG                 = 'WLF'
TAG_EXT             = 'Wiki Loves Folk'

BASE_WLF_NAME       = 'Commons:{1}/{0}'.format(YEAR, TAG_EXT)
LOG_PAGE            = BASE_WLF_NAME + '/Log'
GALLERY_PAGE        = BASE_WLF_NAME + '/User Gallery'

commons_site = pb.Site('commons', 'commons')

In [ ]:
gallery_template = """== Photographs by ${author} in ${tag} ${year} ==

<gallery mode="packed" heights="150px">
% for item in image_list :
${item}
% endfor
</gallery>"""

In [ ]:
reduced_gallery_template = """== Photographs by ${author} in ${tag} ${year} ==

<gallery mode="packed" heights="150px">
% for item in image_list[:10] :
${item}
% endfor
</gallery>
% if len(image_list) > 10:

; See '''gallery''' with all contributions by [[User:${author}|${author}]] '''[[Commons:${tag}/${year}/Images from ${tag} ${year} in Spain by ${author}|here]]'''.
% endif"""

In [ ]:
def sort_gallery(images):
    images = sorted(images)
    return images

In [ ]:
pb.output('Retrieving --> {1} {0} in Spain images list from cache'.format(YEAR, TAG))
list_page = pb.Page(commons_site, LOG_PAGE)
list_page_text = StringIO(list_page.text[list_page.text.find('\n') + 1:list_page.text.rfind('\n')])
images_df = pd.read_csv(list_page_text,
                            sep=";",
                            index_col=False,
                            names=['image_title', 'wikidata_id', 'uploader', 'time_to_upload', 'timestamp', 'qi']
                       ).fillna('')
pb.output('Retrieved --> {1} {0} in Spain images list from cache'.format(YEAR, TAG))

images_df['timestamp'] = pd.to_datetime(images_df['timestamp'], format="%Y-%m-%d %H:%M:%S")

images_df.set_index(["timestamp"], inplace=True)
del images_df.index.name

In [ ]:
grouped_images = images_df.groupby(['uploader'])['image_title'].apply(list).apply(lambda x: sort_gallery(x))
for index, item in grouped_images.iteritems() :
    vars = {
        "author": index,
        "image_list": item,
        "tag": TAG_EXT,
        "year": YEAR
    }
    t = Template(gallery_template)
    gallery_text = t.render(**vars)
    
    gallery_page = pb.Page(commons_site, 'Commons:{0}/{1}/Images from {0} {1} in Spain by {2}'.format(TAG_EXT, YEAR, index))
    if gallery_page.text != gallery_text :
        gallery_page.text = gallery_text
    
        pb.output('Publishing --> {1} {0} in Spain User Gallery'.format(YEAR, TAG))
        gallery_page.save("{1} {0} in Spain User Gallery".format(YEAR, TAG))

In [ ]:
grouped_images = images_df.groupby(['uploader'])['image_title'].apply(list).apply(lambda x: sort_gallery(x))
for index, item in grouped_images.iteritems() :
    vars = {
        "author": index,
        "image_list": item,
        "tag": TAG_EXT,
        "year": YEAR
    }
    t = Template(reduced_gallery_template)
    gallery_text = t.render(**vars)
    
    gallery_page = pb.Page(commons_site, 'Commons:{0}/{1}/Images from {0} {1} in Spain by {2} (reduced)'.format(TAG_EXT, YEAR, index))
    if gallery_page.text != gallery_text :
        gallery_page.text = gallery_text
    
        pb.output('Publishing --> {1} {0} in Spain reduced User Gallery'.format(YEAR, TAG))
        gallery_page.save("{1} {0} in Spain reduced User Gallery".format(YEAR, TAG))

In [ ]:
gallery_index_template = """'''Contributions to [[Commons:${tag}|${tag} ${year}]] in Spain'''

${galleries}

[[Category:${tag} ${year} in Spain]]"""

vars = {
    "galleries": '\n'.join(['{{{{Commons:{0}/{1}/Images from {0} {1} in Spain by {2} (reduced)}}}}'.format(TAG_EXT, YEAR, i) for i in images_df['uploader'].unique()]),
    "tag": TAG_EXT,
    "year": YEAR
}
t = Template(gallery_index_template)
gallery_index_text = t.render(**vars)

In [ ]:
gallery_index_page = pb.Page(commons_site, GALLERY_PAGE)
gallery_index_page.text = gallery_index_text
    
pb.output('Publishing --> {1} {0} in Spain User Gallery'.format(YEAR, TAG))
gallery_index_page.save("{1} {0} in Spain User Gallery".format(YEAR, TAG))